 Importing the Dependencies

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam


 Data Collection and Processing

In [2]:
# loading the csv data to a Pandas DataFrame
heart_data = pd.read_csv('heart.csv')


In [3]:
# Checking the first 5 rows of the dataset
heart_data.head()


,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,0,145,233,1,2,150,0,2.3,2,0,2,0
1,67,1,3,160,286,0,2,108,1,1.5,1,3,1,1
2,67,1,3,120,229,0,2,129,1,2.6,1,2,3,1
3,37,1,2,130,250,0,0,187,0,3.5,2,0,1,0
4,41,0,1,130,204,0,2,172,0,1.4,0,0,1,0


In [4]:
# Checking for missing values
heart_data.isnull().sum()


age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
slope       0
ca          0
thal        0
target      0
dtype: int64

 Splitting the Features and Target

In [5]:
X = heart_data.drop(columns='target', axis=1)
Y = heart_data['target']


In [6]:
# Normalizing the features
scaler = StandardScaler()
X = scaler.fit_transform(X)


 Splitting the Data into Training Data & Test Data

In [7]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=2)


In [8]:
print(X.shape, X_train.shape, X_test.shape)


(303, 13) (242, 13) (61, 13)


 Building the SOCNN-CMM Model

In [9]:
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dropout(0.2),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])


c:\Users\ranga\anaconda3\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


 Model Training

In [10]:
# Training the model
history = model.fit(X_train, Y_train, epochs=50, batch_size=16, validation_split=0.1, verbose=1)


Epoch 1/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.6157 - loss: 0.6530 - val_accuracy: 0.6400 - val_loss: 0.6280
Epoch 2/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7571 - loss: 0.5760 - val_accuracy: 0.6800 - val_loss: 0.5941
Epoch 3/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7900 - loss: 0.5320 - val_accuracy: 0.6800 - val_loss: 0.5722
Epoch 4/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8424 - loss: 0.4432 - val_accuracy: 0.7200 - val_loss: 0.5595
Epoch 5/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8135 - loss: 0.4281 - val_accuracy: 0.7200 - val_loss: 0.5548
Epoch 6/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8307 - loss: 0.4167 - val_accuracy: 0.7200 - val_loss: 0.5552
Epoch 7/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8922 - loss: 0.3088 - val_accuracy: 0.7200 - val_loss: 0.5646
Epoch 8/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8510 - loss: 0.3512 - val_accuracy: 0.7200 - val_loss

 Model Evaluation

In [11]:
# Evaluate the model on training data
train_loss, train_accuracy = model.evaluate(X_train, Y_train, verbose=0)
print(f'Accuracy on Training Data: {train_accuracy:.2f}')

# Evaluate the model on test data
test_loss, test_accuracy = model.evaluate(X_test, Y_test, verbose=0)
print(f'Accuracy on Test Data: {test_accuracy:.2f}')


Accuracy on Training Data: 0.92
Accuracy on Test Data: 0.82


 Building a Predictive System

In [12]:
input_data = (62, 0, 0, 140, 268, 0, 0, 160, 0, 3.6, 0, 2, 2)

# Convert the input data to a numpy array
input_data_as_numpy_array = np.asarray(input_data)

# Reshape the numpy array as we are predicting for only one instance
input_data_reshaped = input_data_as_numpy_array.reshape(1, -1)

# Normalize the input data
input_data_normalized = scaler.transform(input_data_reshaped)

# Make a prediction
prediction = model.predict(input_data_normalized)
if prediction[0] > 0.5:
    print('The Person has Heart Disease')
else:
    print('The Person does not have Heart Disease')


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
The Person does not have Heart Disease


c:\Users\ranga\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


 Saving the Trained Model

In [13]:
# Saving the model
model.save('heart_disease_socnn_model.h5')


 Loading the Saved Model

In [14]:
from tensorflow.keras.models import load_model

# Load the model
loaded_model = load_model('heart_disease_socnn_model.h5')


In [15]:
# Checking feature names
for column in heart_data.columns[:-1]:
    print(column)

age
sex
cp
trestbps
chol
fbs
restecg
thalach
exang
oldpeak
slope
ca
thal
